# 🔐 Autoencoder - Detección de Anomalías y Reducción Dimensional
## Red Neuronal para Comprimir y Detectar Patrones Atípicos

---

### Objetivos:
1. Construir Autoencoder para reducción dimensional
2. Detectar anomalías mediante reconstruction error
3. Visualizar espacio latente con t-SNE
4. Identificar delitos atípicos

**Autor**: Adonnay Bazaldua  
**Fecha**: Noviembre 2025

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import pickle
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
tf.random.set_seed(42)

print(f"✅ TensorFlow version: {tf.__version__}")

In [ ]:
# Cargar datos
X_train = np.load('processed_data/X_train.npy')
X_test = np.load('processed_data/X_test.npy')

with open('processed_data/metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

input_dim = X_train.shape[1]
print(f"✅ Datos cargados: {X_train.shape}, features: {input_dim}")

## Construcción del Autoencoder

### Arquitectura:
```
Encoder:  Input(features) → Dense(128) → Dense(64) → Dense(32) [latent]
Decoder:  Dense(32) → Dense(64) → Dense(128) → Dense(features)
```

In [ ]:
# Encoder
encoder_input = layers.Input(shape=(input_dim,))
encoded = layers.Dense(128, activation='relu')(encoder_input)
encoded = layers.Dense(64, activation='relu')(encoded)
latent = layers.Dense(32, activation='relu', name='latent_space')(encoded)

encoder = models.Model(encoder_input, latent, name='encoder')

# Decoder
decoder_input = layers.Input(shape=(32,))
decoded = layers.Dense(64, activation='relu')(decoder_input)
decoded = layers.Dense(128, activation='relu')(decoded)
decoder_output = layers.Dense(input_dim, activation='linear')(decoded)

decoder = models.Model(decoder_input, decoder_output, name='decoder')

# Autoencoder completo
autoencoder_input = layers.Input(shape=(input_dim,))
encoded_out = encoder(autoencoder_input)
decoded_out = decoder(encoded_out)

autoencoder = models.Model(autoencoder_input, decoded_out, name='autoencoder')

autoencoder.compile(optimizer='adam', loss='mse', metrics=['mae'])

print("\n📊 Encoder:")
encoder.summary()
print("\n📊 Decoder:")
decoder.summary()
print(f"\n📊 Autoencoder completo: {autoencoder.count_params():,} parámetros")

In [ ]:
# Entrenar (entrada = salida deseada)
print("🚀 Entrenando Autoencoder...\n")

history = autoencoder.fit(
    X_train, X_train,  # Input = Output
    epochs=50,
    batch_size=256,
    validation_split=0.2,
    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=1
)

print("\n✅ Entrenamiento completado")

## Detección de Anomalías

Delitos con alto reconstruction error son "anómalos"

In [ ]:
# Reconstruir test set
X_test_reconstructed = autoencoder.predict(X_test, verbose=0)

# Calcular reconstruction error (MSE por muestra)
reconstruction_error = np.mean(np.square(X_test - X_test_reconstructed), axis=1)

# Definir threshold: media + 2*std
threshold = reconstruction_error.mean() + 2 * reconstruction_error.std()

anomalies = reconstruction_error > threshold
num_anomalies = anomalies.sum()

print(f"\n🔍 Detección de Anomalías:")
print(f"   Threshold: {threshold:.6f}")
print(f"   Anomalías detectadas: {num_anomalies} ({num_anomalies/len(X_test)*100:.2f}%)")

# Visualizar distribución de errores
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(reconstruction_error, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {threshold:.4f}')
plt.xlabel('Reconstruction Error (MSE)')
plt.ylabel('Frecuencia')
plt.title('Distribución de Reconstruction Error')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(range(len(reconstruction_error)), reconstruction_error, 
            c=anomalies, cmap='coolwarm', s=10, alpha=0.6)
plt.axhline(threshold, color='red', linestyle='--', linewidth=2)
plt.xlabel('Índice de Muestra')
plt.ylabel('Reconstruction Error')
plt.title('Anomalías Detectadas (Rojo)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('models/autoencoder_anomalies.png', dpi=300)
plt.show()

## Visualización del Espacio Latente con t-SNE

In [ ]:
# Obtener representación en espacio latente
latent_representation = encoder.predict(X_test[:5000], verbose=0)  # Muestra para velocidad

# Aplicar t-SNE para reducir a 2D
print("🔄 Aplicando t-SNE (esto puede tardar)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
latent_2d = tsne.fit_transform(latent_representation)

# Visualizar
y_test = np.load('processed_data/y_test.npy')

plt.figure(figsize=(12, 10))
scatter = plt.scatter(latent_2d[:, 0], latent_2d[:, 1], 
                      c=y_test[:5000], cmap='tab20', s=10, alpha=0.6)
plt.colorbar(scatter, label='Clase de Delito')
plt.xlabel('t-SNE Dimensión 1')
plt.ylabel('t-SNE Dimensión 2')
plt.title('Visualización del Espacio Latente (32D → 2D)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('models/autoencoder_tsne.png', dpi=300)
plt.show()

print("✅ Visualización completada")

In [ ]:
# Guardar modelos
autoencoder.save('models/autoencoder_final.keras')
encoder.save('models/encoder.keras')
decoder.save('models/decoder.keras')

# Guardar resultados
with open('models/autoencoder_results.pkl', 'wb') as f:
    pickle.dump({
        'threshold': threshold,
        'num_anomalies': int(num_anomalies),
        'anomaly_percentage': float(num_anomalies/len(X_test)*100),
        'mean_reconstruction_error': float(reconstruction_error.mean()),
        'num_parameters': autoencoder.count_params()
    }, f)

print("\n✅ Autoencoder guardado")
print("\n📝 Próximo paso: Análisis comparativo de todos los modelos")
print("   → Notebook: 07_Comparative_Analysis.ipynb")